<a href="https://colab.research.google.com/github/vijjus/ST-summer-2024/blob/main/Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/vijjus/ST-summer-2024.git

In [ ]:
!ls

In [ ]:
# we will be using the following packages for our work today
import dlib
import cv2
import imutils
import matplotlib.pyplot as plt

In [ ]:
# If you don't see an output from this command, click Runtime -> Change runtime type
# Pick V100
# Runtime should prompt to restart
!nvidia-smi

Facial recognition - use a model to detect and identify a face.

In [ ]:
image1 = cv2.imread("ST-summer-2024/family.jpeg", cv2.IMREAD_COLOR)

In [ ]:
# resize - mainly needed for smaller images
image1 = imutils.resize(image1, width=600)

In [ ]:
# color images have 3 channels - Red (R), Blue (B), Green (G)
# change from BGR to RGB format (OpenCV loads images in BGR format)
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(image1)

In [ ]:
# This is the first detector we are going to try call HOG + SVM
# Histogram of Gradients + Support Vector Machine
detector = dlib.get_frontal_face_detector()

In [ ]:
# This detector is good at detecting frontal faces
# XXX: Try detecting a sideways facing face with this detector
rects = detector(image1)

In [ ]:
# convenience function to take co-ordinates and convert them to bbox format
# bbox = Bounding Box (used to mark sections of an image)
def convert_and_trim_bb(image, rect):
	# extract the starting and ending (x, y)-coordinates of the
	# bounding box
	startX = rect.left()
	startY = rect.top()
	endX = rect.right()
	endY = rect.bottom()
	# ensure the bounding box coordinates fall within the spatial
	# dimensions of the image
	startX = max(0, startX)
	startY = max(0, startY)
	endX = min(endX, image.shape[1])
	endY = min(endY, image.shape[0])
	# compute the width and height of the bounding box
	w = endX - startX
	h = endY - startY
	# return our bounding box coordinates
	return (startX, startY, w, h)

In [ ]:
# convert the resulting dlib rectangle objects to bounding boxes,
# then ensure the bounding boxes are all within the bounds of the
# input image
boxes = [convert_and_trim_bb(image1, r) for r in rects]

In [ ]:
# the boxes are the X, Y (top left, width & height)
boxes

In [ ]:
# loop over the bounding boxes
for (x, y, w, h) in boxes:
	# draw the bounding box on our image
	cv2.rectangle(image1, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(image1, cmap="gray")

In [ ]:
# Now we will try a Concolutional Neural Network based detector
detector2 = dlib.cnn_face_detection_model_v1("/content/mmod_human_face_detector.dat")

In [ ]:
image2 = cv2.imread("ST-summer-2024/family.jpeg", cv2.IMREAD_COLOR)

In [ ]:
# resize - mainly needed for smaller images
image2 = imutils.resize(image2, width=600)

In [ ]:
# color images have 3 channels - Red (R), Blue (B), Green (G)
# change from BGR to RGB format (OpenCV loads images in BGR format)
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

In [ ]:
# this model is running on the GPU
results = detector2(image2)

In [ ]:
# convert the resulting dlib rectangle objects to bounding boxes,
# then ensure the bounding boxes are all within the bounds of the
# input image
boxes = [convert_and_trim_bb(image2, r.rect) for r in results]

In [ ]:
# loop over the bounding boxes
for (x, y, w, h) in boxes:
	# draw the bounding box on our image
	cv2.rectangle(image2, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(image2)

In [ ]:
def detect_faces(detector, image_path, is_CNN=False):
  """
  @brief  Takes in a detector and a path to an image, and performs
          detects human faces.
  """
  image = cv2.imread(image_path, cv2.IMREAD_COLOR)
  image = imutils.resize(image, width=600)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  rects = detector(image)

  if is_CNN:
    boxes = [convert_and_trim_bb(image, r) for r in rects]
  else:
    boxes = [convert_and_trim_bb(image, r.rect) for r in rects]

  for (x, y, w, h) in boxes:
	  # draw the bounding box on our image
	  cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

  plt.figure(figsize=(10,20))
  plt.imshow(image)

In [ ]:
detect_faces(detector, "ST-summer-2024/45183364-young-people-browsing-the-internet.jpg")

In [ ]:
detect_faces(detector2, "ST-summer-2024/45183364-young-people-browsing-the-internet.jpg")